In [1]:
from src.classifier import Classifier
from src.utils import count_parameters, load_weights

model = Classifier()

device = "cuda:4"
model.to(device)

load_weights(model, "out/accelerating_training/best_model.pt")

/home/mavrinti/accelerating_train/src/utils.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_state_dict = th.load(weights_path, map_location="cpu")


In [2]:
model

Classifier(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

In [3]:
num_params, top = count_parameters(model, return_top=True)
print("num params:", num_params, "top:", top)

num params: 41626 top: [(18432, 'conv4.weight'), (16384, 'fc1.weight'), (4608, 'conv3.weight'), (1152, 'conv2.weight'), (640, 'fc2.weight')]


In [4]:
from src.dataloader import load_cifar

_, eval_loader = load_cifar(train_batch_size=64, eval_batch_size=64)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
import torch as th
import numpy as np
from sklearn.metrics import accuracy_score

def eval(model, device, eval_loader):
    labels_true = []
    labels_pred = []
    for step, (data, label) in enumerate(eval_loader, start=1):
        data = data.to(device)

        with th.no_grad():
            label_pred = model(data).argmax(axis=-1).squeeze()

        labels_true.append(label.numpy())
        labels_pred.append(label_pred.cpu().numpy())
        
    score = accuracy_score(np.concatenate(labels_true), np.concatenate(labels_pred))
    return score

In [6]:
eval(model, device, eval_loader)

0.7182

In [7]:
from src.decomposition import tucker_decompose_model
import tensorly as tl

tl.set_backend('pytorch')
model = tucker_decompose_model(model, linear_max_rank=50, conv_max_rank=55)

/home/mavrinti/.venv/lib/python3.10/site-packages/tensorly/decomposition/_tucker.py:167: Warning: Given only one int for 'rank' instead of a list of 2 modes. Using this rank for all modes.
  warnings.warn(message, Warning)


In [8]:
model

Classifier(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Sequential(
    (0): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): Conv2d(32, 55, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (2): Conv2d(55, 64, kernel_size=(1, 1), stride=(1, 1))
  )
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Sequential(
    (0): Linear(in_features=256, out_features=50, bias=False)
    (1): Linear(in_features=50, out_features=64, bias=True)
  )
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

In [9]:
num_params, top = count_parameters(model, return_top=True)
print("num params:", num_params, "top:", top)

num params: 43194 top: [(15840, 'conv4.1.weight'), (12800, 'fc1.0.weight'), (4608, 'conv3.weight'), (3520, 'conv4.2.weight'), (3200, 'fc1.1.weight')]


In [10]:
eval(model, device, eval_loader)

0.6758

In [11]:
model = Classifier()
model.to(device)
load_weights(model, "out/accelerating_training/best_model.pt")

/home/mavrinti/accelerating_train/src/utils.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_state_dict = th.load(weights_path, map_location="cpu")


In [12]:
model = tucker_decompose_model(model, linear_max_rank=8, conv_max_rank=8)

/home/mavrinti/.venv/lib/python3.10/site-packages/tensorly/decomposition/_tucker.py:167: Warning: Given only one int for 'rank' instead of a list of 2 modes. Using this rank for all modes.
  warnings.warn(message, Warning)


In [13]:
model

Classifier(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Sequential(
    (0): Conv2d(8, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
  )
  (conv3): Sequential(
    (0): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
  )
  (conv4): Sequential(
    (0): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (2): Conv2d(8, 64, kernel_size=(1, 1), stride=(1, 1))
  )
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Sequential(
    (0): Linear(in_features=256, out_features=8, bias=False)
    (1): Linear(in_features

In [14]:
num_params, top = count_parameters(model, return_top=True)
print("num params:", num_params, "top:", top)

num params: 6634 top: [(2048, 'fc1.0.weight'), (576, 'conv2.1.weight'), (576, 'conv3.1.weight'), (576, 'conv4.1.weight'), (512, 'conv4.2.weight')]


In [15]:
eval(model, device, eval_loader)

0.1217

---